# State transfer with filters

This notebook analyzes the state transfer protocols from [Guillermo F. Peñas et al (2021)](https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.17.054038), extending them to work with Purcell filters. The presence of the filter changes the shape of the ideal controls.

The notebook illustrates the following aspects from the library:

1. Creation of a model class that describes the ETH setup with two cavities, two Purcell filters and two qubits.
2. Creation of initial conditions for the simulation, with a single excited qubit.
3. Simulation of the evolution of the link, as we switch on and off the coupling of both qubits.
4. Comparing the performances of state transfer of two possible qubit-cavity controls, designed with- and without Purcell filters.

## Setup

While the SuperQuLAN library can be installed, this notebook only assumes that it resides inside the `examples` folder and that the library is located under `../superqulan`.

In [1]:
import sys
if "../" not in sys.path:
    sys.path = ["../"] + sys.path

In [2]:
import numpy as np 
from math import pi as π
from superqulan.waveguide import Waveguide
from superqulan.simulator import Trotter_solver_dynamics
from superqulan.architecture import Exp_2qubits_2cavities_2purcells

In [3]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('agg')
plt.rc('font', size=20)

## Control design

### Qubit-cavity-link setup

If we considered a system consisting of just an emitter, a resonator and a decay channel, it can be modeled as a two-level system, with the only excitation residing in the qubit, in the cavity or moving away in the waveguide:
\begin{align}
    \dot{q}(t)&=-i g(t) c(t)\\
    \dot{c}(t)&=-i g(t) q(t)-\frac{\kappa}{2}c(t),
\end{align}
This last case is modeled using input-output theory by means of the cavity decay rate into the waveguide $\kappa$.

Our goal is to produce an outgoing wavepacket of the form
\begin{align}
    \psi(t)=-\sqrt{\kappa/4}{\rm sech}(\kappa t/2).
\end{align}

Using input-output theory this can be achieved exactly by the control
$$ g(t) = \frac{\kappa}{2} \text{sech}\left(\kappa t /2\right) $$

In [4]:
def gt_sech(t, κ = 2*π*20e-3 ):
    return κ / 2 / np.cosh(κ*t/2)

### Qubit-cavity-filter-link setup

If instead one considers that there is a Purcell filter in between the resonator and the waveguide as it is the case in the ETH setup, the dynamical equations are slightly different
\begin{align}
    \dot{q}(t)&=-ig(t) c(t)\\
    \dot{c}(t)&=-ig(t) q(t) -ig_p b(t)\\
    \dot{b}(t)&=-ig_p c(t)-\kappa/2 b(t).
\end{align}
Now the outgoing photon is controled by the amplitude of the wavefunction in the Purcell filter $b(t)$.

Using input-output theory we once more find that the control that produces a sech-pulse photon has an analytical solution of the form
\begin{align}
    g(t)=\frac{{\rm sech}^3(\kappa t/(2\eta))\left(4g_p^2\eta^2-3\kappa^2+(4g_p^2\eta^2+\kappa^2)\cosh(\kappa t/\eta)-\eta\kappa^2\sinh(\kappa t/\eta) \right)}{4\eta\left(\kappa^2{\rm sech}^4(\kappa t/(2\eta))-8 g_p^2\eta^3(\tanh(\kappa t/(2\eta))-1)-{\rm sech}^2(\kappa t/(2\eta))\left(\kappa^2+\eta^2(4g_p^2+\kappa^2)-2\eta\kappa^2\tanh(\kappa t/(2\eta)) \right)\right)^{1/2}}.
\end{align}

In [5]:
def gt(t, κ=2 * π * 20e-3, η=6.76, g_p=2 * π * 25e-3):
    return (
        ((1 / np.cosh(κ * t / (2 * η))) ** 3)
        * (
            4 * g_p**2 * η**2
            - 3 * κ**2
            + (4 * g_p**2 * η**2 + κ**2) * np.cosh(κ * t / η)
            - η * κ**2 * np.sinh(κ * t / η)
        )
    ) / (
        4
        * η
        * (
            κ**2 * ((1 / np.cosh(κ * t / (2 * η))) ** 4)
            - 8 * g_p**2 * η**3 * (np.tanh(κ * t / (2 * η)) - 1)
            - ((1 / np.cosh(κ * t / (2 * η))) ** 2)
            * (
                κ**2
                + η**2 * (4 * g_p**2 + κ**2)
                - 2 * η * κ**2 * np.tanh(κ * t / (2 * η))
            )
        )
        ** 0.5
    )


### Comparison

The controls differ: when the cavity is directly connected to the waveguide, the qubit-cavity coupling canbe switched off at the end of the photon generation process. If there is a Purcell filter, this is not the case. This is best seen in the following plot.

In [6]:
fig, (ax) = plt.subplots(nrows=1, figsize=(8,6), sharex=True)
κ_eff = 2*π*130e-3/6.76
t = np.linspace(-30/κ_eff, 30/κ_eff, 1000)
ax.plot(t*κ_eff, gt_sech(t)/κ_eff, label='simple')
ax.plot(t*κ_eff, gt(t, κ = 2*π*130e-3)/κ_eff, label='Purcell')
ax.set_xlabel('t * $\kappa_{eff}$')
ax.set_ylabel('$gt / \kappa_{eff}$')
ax.legend()
fig.savefig('figures/fig-02-001.svg')

![Fig. 1](./figures/fig-02-001.svg)

## State transfer experiment

We now repeat the state-transfer theory from [Guillermo F. Peñas et al (2021)](https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.17.054038), but simulating a setup in which we have two qubits and two cavities at the sides of the quantum link, but the cavities join the link via two Purcell filters. Our simulation is based on a relatively flexible function, that allows for different controls `gt`, slowing down of the controls via an enlarging factor `η`, varying the setup dimensions, frequencies, etc.

In [7]:
def state_transfer(
    improved_control=False,
    lamb_shift=1,
    δ_qubit=2 * π * 8.40701933547913,
    ω_cavity=2 * π * 8.40701933547913,
    ω_filter=2 * π * 8.40701933547913,
    g_p=2 * π * 25e-3,
    kappa=2 * π * 20e-3,
    η=1,
    pulse_size=10,
    steps=3000,
    length=30,
    modes=2101,
    tdelay=None,
    quiet=False,
    filename=None,
):

    link = Waveguide(frequency=ω_cavity + lamb_shift, modes=modes, length=length)

    if tdelay is None:
        # separation between protocol g_1(t) and g_2(t) /time reversed.
        tdelay = link.tprop

    duration = tdelay + 2 * pulse_size * (η / kappa)
    time = np.linspace(-duration / 2, duration / 2, steps)

    if improved_control:

        def control(t):
            return gt(t, κ=kappa, η=η, g_p=g_p)

    else:
        κ_eff = 4 * g_p**2 / kappa

        def control(t):
            return gt_sech(t, κ=κ_eff)

    setup = Exp_2qubits_2cavities_2purcells(
        δ1=δ_qubit,
        δ2=δ_qubit,
        ω1=ω_cavity,
        ω2=ω_cavity,
        ωp1=ω_filter,
        ωp2=ω_filter,
        g1=lambda t: control(t + tdelay / 2),
        g2=lambda t: control(tdelay / 2 - t),
        gp1=g_p,
        gp2=g_p,
        κ1=kappa,
        κ2=kappa,
        δLamb=lamb_shift,
        waveguide=link,
    )

    if not quiet:
        print("Lamb_shift=", setup.E[0] - setup.δ1)
        print("Frequency of qubit 1=", setup.E[0])
        print(f"Initial value g_1(-tf) = (2π)*{setup.g1(time[0]-tdelay/2)/2/π*1e3}MHz")
        print(f"kappa = (2π) {kappa / 2 / π * 1e3} MHz")
        print(f"η = {η}")
        print(f"g_p = (2π) {g_p / 2 / π * 1e3} MHz")
        print(f"Total time = {duration/link.tprop} x Propagation_time")

    vt = Trotter_solver_dynamics(time, setup.excited_qubit(which=0), setup.Hamiltonian)
    P = setup.mode_occupations(vt)
    Pwaveguide = np.sum(P[setup.waveguide_indices, :], 0)
    if not quiet:
        print(
            f"|q_1(tf)|^2 = {P[0,-1]}\n"
            f"|c_1(tf)|^2 = {P[2,-1]}\n"
            f"|filter_1(tf)|^2 = {P[4,-1]}\n"
            f"|wv(tf)|^2  = {Pwaveguide[-1]} (max = {max(Pwaveguide)})\n"
            f"|filter_2(tf)|^2 = {P[5,-1]}\n"
            f"|c_2(tf)|^2 = {P[3,-1]}\n"
            f"|q_2(tf)|^2 = {P[1,-1]}\n"
            f" 1-F        = {1-P[1,-1]}"
        )
    if filename:
        fig, (ax, axc) = plt.subplots(nrows=2, figsize=(10, 8), sharex=True)
        ax.plot(time * kappa, P[setup.qubit_indices[0], :], label="qubit 1")
        ax.plot(time * kappa, P[setup.cavity_indices[0], :], "--", label="cavity 1")
        ax.plot(time * kappa, P[setup.filter_indices[0], :], "--", label="Purcell 1")

        ax.plot(time * kappa, Pwaveguide, "-.", label="waveguide")

        ax.plot(time * kappa, P[setup.filter_indices[1], :], "--", label="Purcell 2")
        ax.plot(time * kappa, P[setup.cavity_indices[1], :], "--", label="cavity 2")
        ax.plot(time * kappa, P[setup.qubit_indices[1], :], label="qubit 2")

        ax.legend(fontsize=16)
        ax.set_title(
            f"$l={length}\,m,\,"
            r"\omega=2\pi\times"
            f"{setup.ω1/(2*π)}"
            r"\,\mathrm{GHz},\,\kappa_filter=2\pi\times"
            f"{np.round(kappa/(2e-3*π))}"
            r"\,\mathrm{MHz},\,\kappa_{eff}="
            f"{np.round(kappa/(2e-3*π)/η,3)}$",
            fontsize=16,
        )
        axc.plot(time * kappa, setup.g1(time) / kappa / η, label=r"$g_1(t)/\kappa_1$")
        axc.plot(time * kappa, setup.g2(time) / kappa / η, label=r"$g_2(t)/\kappa_2$")
        axc.set_xlabel(r"$t\kappa_1$")
        axc.set_ylabel(r"$g_i(t)/\kappa_i$")
        axc.legend(fontsize=16)
        fig.tight_layout()
        fig.savefig(filename)

    timeres = time / link.tprop
    q1end = P[setup.qubit_indices[0], -1]
    q2end = P[setup.qubit_indices[0], -1]
    return q2end, q1end, vt, duration, time, timeres, link.frequencies


## Comparison plots

In [8]:
state_transfer(
    improved_control=False,
    lamb_shift=(0.0229) * (2 * π) * 19.231e-3,
    δ_qubit=2 * π * 8.40701933547913,
    ω_cavity=2 * π * 8.40701933547913,
    ω_filter=2 * π * 8.40701933547913,
    g_p=2 * π * 25e-3,
    kappa=2 * π * 130e-3,
    η=6.76,
    pulse_size=10,
    steps=3000,
    length=30,
    modes=2101,
    quiet=False,
    filename='./figures/fig-02-002.svg',
);


Lamb_shift= 0.002767051349110261
Frequency of qubit 1= 52.82562741720627
Initial value g_1(-tf) = (2π)*0.0010346975532944597MHz
kappa = (2π) 130.0 MHz
η = 6.76
g_p = (2π) 25.0 MHz
Total time = 2.035162895299994 x Propagation_time
|q_1(tf)|^2 = 6.522227499278327e-05
|c_1(tf)|^2 = 1.3054623283892215e-08
|filter_1(tf)|^2 = 6.26772187737567e-08
|wv(tf)|^2  = 0.00466717422996842 (max = 0.9999246704673552)
|filter_2(tf)|^2 = 1.382006067649532e-05
|c_2(tf)|^2 = 8.340064959590051e-06
|q_2(tf)|^2 = 0.9952453676372466
 1-F        = 0.004754632362753419


The plot below shows the populations of the qubits, cavity and waveguide modes, running the original 'sech' protocol. Note how initially the qubit 1 is excited and it progressively transfers its population to the cavity. The photon then leaks from the cavity into the waveguide, and is absorbed by the second cavity and ultimately transferred to the second qubit.

![Fig. 2](./figures/fig-02-002.svg)

In [9]:
state_transfer(
    improved_control=True,
    lamb_shift=(0.0229) * (2 * π) * 19.231e-3,
    δ_qubit=2 * π * 8.40701933547913,
    ω_cavity=2 * π * 8.40701933547913,
    ω_filter=2 * π * 8.40701933547913,
    g_p=2 * π * 25e-3,
    kappa=2 * π * 130e-3,
    η=6.76,
    pulse_size=10,
    steps=3000,
    length=30,
    modes=2101,
    quiet=False,
    filename='./figures/fig-02-003.svg',
);


Lamb_shift= 0.002767051349110261
Frequency of qubit 1= 52.82562741720627
Initial value g_1(-tf) = (2π)*0.0011112284366120335MHz
kappa = (2π) 130.0 MHz
η = 6.76
g_p = (2π) 25.0 MHz
Total time = 2.035162895299994 x Propagation_time
|q_1(tf)|^2 = 4.5796986634352227e-07
|c_1(tf)|^2 = 8.84820394792789e-08
|filter_1(tf)|^2 = 6.309576706322417e-08
|wv(tf)|^2  = 0.0002541015515988493 (max = 0.9999306556058178)
|filter_2(tf)|^2 = 6.106032359152791e-06
|c_2(tf)|^2 = 8.014267107082724e-05
|q_2(tf)|^2 = 0.9996590401969907
 1-F        = 0.00034095980300929973


The plots change in a subtle way when we use the new control. The population exchange looks similar, but the excitation of the second qubit is more complete, as evidenced by the one order of magnitude decrease of the fidelity. Also, the controls are different, as shown above.

![Fig. 3](./figures/fig-02-003.svg)

## Systematic computations

In the following simulations we compute the fidelity for increasing pulse durations (i.e. truncating each time further away on the tails of the distribution) and plot the results for the two controls. As shown in the plot, the control that takes into account the Purcell filter improves several orders of magnitude the fidelity.

In [10]:
q2end = [[],[]]
for run in [0,1]:
    Protocol_time = []
    for i in np.linspace(1, 16, 15):
        q2end_i, _, _, Protocol_time_i, _, _, _ = state_transfer(
            improved_control=(run == 1),
            lamb_shift=(0.0229) * (2 * π) * 19.231e-3,
            δ_qubit=2 * π * 8.40701933547913,
            ω_cavity=2 * π * 8.40701933547913,
            ω_filter=2 * π * 8.40701933547913,
            g_p=2 * π * 25e-3,
            kappa=2 * π * 130e-3,
            η=6.76,
            pulse_size=i,
            steps=3000,
            length=30,
            modes=2101,
            quiet=True,
            filename=None,
        )
        q2end[run].append(q2end_i)
        Protocol_time.append(Protocol_time_i)


In [11]:
fig, (ax) = plt.subplots(nrows=1, figsize=(8,4))
ax.plot(np.array(Protocol_time), 1-np.array(q2end[0]), label = 'Sech control')
ax.plot(np.array(Protocol_time), 1-np.array(q2end[1]), label = 'Exact control')

ax.semilogy()
ax.axhline(y=1.2e-5, linestyle = ':')
ax.legend()
ax.set_xlabel('$t(ns)$')
ax.set_ylabel('$1-Q2(T)$')
fig.tight_layout()

fig.savefig('./figures/fig-02-004.svg')

The plot below shows the errors in the state transfer, measured by the different between the second qubit having 100% occupation and the outcome of the simulation.

![Fig. 4](./figures/fig-02-004.svg)